**现需要制定优惠活动(满X元减Y元，整体优惠额度为5%)，优惠券设计为3个梯级，
为了领券和转化效果最大化，请给出3个梯级的提案，并给出数据分析和方案设计的思路与过程**

In [165]:
import pandas as pd
pd.options.mode.chained_assignment = None
import datetime
import numpy as np
import seaborn as sns
import statistics
import matplotlib.pyplot as plt
from scipy.stats.stats import pearsonr  

<ipython-input-165-de1f1cc29804>:8: DeprecationWarning: Please use `pearsonr` from the `scipy.stats` namespace, the `scipy.stats.stats` namespace is deprecated.
  from scipy.stats.stats import pearsonr


In [35]:
# read xls file 
xls = pd.ExcelFile(r"数据分析数据源_脱敏_20230324.xls")
sheetxls = xls.parse(0)
sheetxls_price = xls.parse(1)


# join the dataframe by vib
temp = sheetxls.join(sheetxls_price.set_index('vib'), on = 'vib')
siemens_dat = temp.loc[sheetxls['brand'] == "西门子"]
siemens_dat = siemens_dat.sort_values(by = ['user_id', 'installation_date'])
siemens_dat = siemens_dat.drop('brand', axis = 1)


# define a dictionary for following steps
user_id = list(siemens_dat['user_id'].unique())
siemens_user_dat = {}
for user in user_id: 
    siemens_user_dat[user] = siemens_dat.loc[siemens_dat['user_id'] == user]
del user


products = siemens_dat['prod_group'].unique()
return_purchase_dict = {}
for prod in products: 
    temp_dict = {}
    for user in siemens_user_dat.keys():
        # extract purchase records for the product and customer
        user_df = siemens_user_dat[user]
        temp_df = user_df.loc[user_df['prod_group'] == prod]
        if not temp_df.empty:
            temp_df = temp_df[['user_id', 'vib', 'price', 'prod_group', 'province', 'installation_date']]
            temp_dict[user] = temp_df
    return_purchase_dict[prod] = temp_dict
del temp_dict, user_df, temp_df


for prod in return_purchase_dict.keys():
    prod_dict = return_purchase_dict[prod]
    for user in prod_dict.keys():
        user_df = prod_dict[user]
        time_stamp_list = list(user_df['installation_date'])
        time_lag = []
        for i in range(len(time_stamp_list)):
            if i == 0:
                time_lag.append(time_stamp_list[i] - time_stamp_list[0])
            else:
                time_lag.append(time_stamp_list[i] - time_stamp_list[i-1])
        user_df['time_lag'] = time_lag
        # filter < 360
        time_1 = datetime.timedelta(days = 360)
        time_2 = datetime.timedelta(days = 0)
        prod_dict[user] = user_df.loc[(user_df['time_lag'] > time_1) | (user_df['time_lag'] == time_2)] 
        prod_dict[user]['time_lag'] = prod_dict[user]['time_lag'].dt.days
    return_purchase_dict[prod] = prod_dict
del prod_dict, user_df, time_stamp_list, time_lag

In [36]:
return_purchase_dict['洗碗机'][0].iloc[0]['price']

5999

**优惠活动分配逻辑:**

在task_A的分析中, 主要从顾客首购后关联购买, 以及同品类复购分析; 其中, 在首购分析中, 主要结论为洗衣机与冰箱的受关联购买的概率较高. 然而, 其中某些品类首购率较低, 分析结果相对不够稳定. 因此, 改优惠活动分配主要考虑从同品类复购角度出发:
- 同类产品中, 购买单价更高的产品是否能带来更好的复购率; 
- 同类产品中, 购买单价更高的产品与复购周期是否有关; 

In [173]:
col_names = np.array(['购买顾客', '购买总销售额', '复购顾客', '复购总量', '复购总销售额'])
indexes = siemens_dat['prod_group'].unique()
return_summary = pd.DataFrame(0, index=indexes, columns=col_names)
del col_names

for prod in return_purchase_dict.keys():
    for user in return_purchase_dict[prod].keys():
        user_df = return_purchase_dict[prod][user]
        return_summary.loc[prod]['购买顾客'] += 1
        return_summary.loc[prod]['购买总销售额'] += user_df.iloc[0]['price']
        rows = user_df.shape[0]
        if rows > 1:
            return_summary.loc[prod]['复购顾客'] += 1
            for i in range(1, rows):
                return_summary.loc[prod]['复购总销售额'] += user_df.iloc[i]['price']
        return_summary.loc[prod]['复购总量'] += (rows - 1)
        
del user_df, rows
return_summary['复购率'] = round(return_summary['复购顾客']/return_summary['购买顾客'], 3)
return_summary['复购率'] = (100.*return_summary['复购率']).round(1).astype(str) + '%'
return_summary['平均复购量'] = round(return_summary['复购总量']/return_summary['复购顾客'], 2)
return_summary['购买单价'] = round(return_summary['购买总销售额']/return_summary['购买顾客'], 2)
return_summary['复购单价'] = round(return_summary['复购总销售额']/return_summary['复购总量'], 2)
return_summary['单价增幅'] = round(return_summary['复购单价'] - return_summary['购买单价'], 2)

In [178]:
return_summary.sort_values(by = '复购率')

,购买顾客,购买总销售额,复购顾客,复购总量,复购总销售额,复购率,平均复购量,购买单价,复购单价,单价增幅
蒸/烤箱,66,534219,9,9,56247,13.6%,1.00,8094.23,6249.67,-1844.56
烘干机,47,421853,7,8,68492,14.9%,1.14,8975.60,8561.50,-414.10
灶具,120,344088,26,27,102301,21.7%,1.04,2867.40,3788.93,921.53
吸油烟机,103,511860,25,28,145793,24.3%,1.12,4969.51,5206.89,237.38
洗碗机,159,932250,65,83,481596,40.9%,1.28,5863.21,5802.36,-60.85
冰箱,220,2089942,136,204,1960467,61.8%,1.50,9499.74,9610.13,110.39
洗衣机,251,1239506,162,245,1242011,64.5%,1.51,4938.27,5069.43,131.16


In [149]:
# ['洗碗机' '洗衣机' '灶具' '冰箱' '蒸/烤箱' '烘干机' '吸油烟机']
# 分析洗碗机特征
def price_ratio(prod):
    prod_dict = return_purchase_dict[prod]
    prod_info = {}
    for usr in prod_dict.keys():
        rows = prod_dict[usr].shape[0]
        price = prod_dict[usr].iloc[0]['price']
        if price in list(prod_info.keys()):
            if rows == 1: 
                prod_info[price] = pd.concat([prod_info[price], prod_dict[usr]])
            else:
                prod_info[price] = pd.concat([prod_info[price], prod_dict[usr].iloc[[0, 1]]])
        else:
            if rows == 1: 
                prod_info[price] = prod_dict[usr]
            else:
                prod_info[price] = prod_dict[usr].iloc[[0, 1]]
    indexes = prod_info.keys()
    col_names = np.array(['购买客户', '复购客户'])
    prod_summary = pd.DataFrame(0, index=indexes, columns=col_names)
    for price in indexes:
        col = prod_info[price]['time_lag']
        prod_summary.loc[price]['购买客户'] = (col == 0).sum()
        prod_summary.loc[price]['复购客户'] = (col != 0).sum()
    prod_summary['复购率'] = round(prod_summary['复购客户']/prod_summary['购买客户'], 2)
    prod_summary = prod_summary.sort_index()
    return prod_info, prod_summary

In [172]:
prod_info, prod_summary = price_ratio('洗碗机')
print('洗碗机:', pearsonr(prod_summary.index, prod_summary['复购率']))
prod_info, prod_summary = price_ratio('洗衣机')
print('洗衣机:', pearsonr(prod_summary.index, prod_summary['复购率']))
prod_info, prod_summary = price_ratio('灶具')
print('灶具:', pearsonr(prod_summary.index, prod_summary['复购率']))
prod_info, prod_summary = price_ratio('冰箱')
print('冰箱:', pearsonr(prod_summary.index, prod_summary['复购率']))
prod_info, prod_summary = price_ratio('蒸/烤箱')
print('蒸/烤箱:', pearsonr(prod_summary.index, prod_summary['复购率']))
prod_info, prod_summary = price_ratio('烘干机')
print('烘干机:', pearsonr(prod_summary.index, prod_summary['复购率']))
prod_info, prod_summary = price_ratio('吸油烟机')
print('吸油烟机:', pearsonr(prod_summary.index, prod_summary['复购率']))

洗碗机: PearsonRResult(statistic=-0.19426606030289614, pvalue=0.35210756451943626)
洗衣机: PearsonRResult(statistic=-0.09784046043679896, pvalue=0.5325109286728456)
灶具: PearsonRResult(statistic=0.5150879090886569, pvalue=0.01189623574261907)
冰箱: PearsonRResult(statistic=0.1060568307472232, pvalue=0.4731015075955131)
蒸/烤箱: PearsonRResult(statistic=-0.09867829571594078, pvalue=0.6621880738096882)
烘干机: PearsonRResult(statistic=0.5984425576743427, pvalue=0.11703960897956721)
吸油烟机: PearsonRResult(statistic=0.050737810027149055, pvalue=0.7976391579259704)


In [180]:
pearsonr(prod_summary.index, prod_summary['复购率'])[0]

0.050737810027149055

**Pearson相关系数结论:**
- 灶具, 烘干机购买单价与后续复购率成强正比关系
- 洗碗机购买单价与后续复购率成弱负比关系
- 冰箱, 洗衣机, 蒸/烤箱, 吸油烟机的购买单价与复购率没有太大关系

### 综合结论

**优惠梯度:**
- 第一梯级(优惠力度大): 灶具, 烘干机 (整体回购率低, 然而当因优惠活动而购买单价更高的产品后, 回购率显著提升; 给予该类产品大幅优惠, 见效明显)
- 第二梯级(优惠力度中等): 冰箱, 洗衣机, 洗碗机 (虽然购买单价的提升对回购率没有显著变化, 然而其本身的回购率相对很高; 中等幅度优惠也有利于更多顾客入手产品, 后续仍有稳定见效)
- 第三梯级(优惠力度较小): 蒸/烤箱, 吸油烟机 (相对优势较小)